In [8]:
import numpy as np
from numpy.typing import NDArray
import pandas as pd

from sklearn import datasets, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from quri_parts.core.state import quantum_state, GeneralCircuitQuantumState
from quri_parts.circuit import inverse_circuit

from scikit_quri.circuit import create_ibm_embedding_circuit, LearningCircuit
from quri_parts.qulacs.sampler import create_qulacs_vector_sampler

In [9]:
n_shots = 1000
sampler = create_qulacs_vector_sampler()

iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
x = df.loc[:, ["petal length (cm)", "petal width (cm)"]]
x_train, x_test, y_train, y_test = train_test_split(
    x, iris.target, test_size=0.25, random_state=0
)
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

In [10]:
n_qubit = 4  # x_train の次元数以上必要。あまり小さいと結果が悪くなる。
circuit = create_ibm_embedding_circuit(n_qubit)

data_states = []
svc = svm.SVC(kernel="precomputed")

In [11]:
def run_circuit(circuit: LearningCircuit, x: NDArray[np.float64]) -> GeneralCircuitQuantumState:
    bound_circuit = circuit.bind_input_and_parameters(x, np.array([]))
    state = quantum_state(n_qubits=n_qubit, circuit=bound_circuit)
    return state

def calc_kernel(bra_data: NDArray[np.float64], ket_data: NDArray[np.float64]) -> float:
    bra_qc = run_circuit(circuit, bra_data).circuit
    ket_qc = inverse_circuit(run_circuit(circuit, ket_data).circuit)
    qc = bra_qc.combine(ket_qc)
    counts = sampler(qc, n_shots)
    count_zero = counts.get(0)
    if count_zero is None:
        return 0.0
    return float(count_zero / n_shots)

def fit(x: NDArray[np.float64], y: NDArray[np.int64]) -> None:
    kar = np.zeros((len(x), len(x)))
    for i in range(len(x)):
        for j in range(len(x)):
            kar[i][j] = calc_kernel(x[i], x[j])

    svc.fit(kar, y)

def predict(xs: NDArray[np.float64], x: NDArray[np.float64]) -> NDArray[np.int64]:
    kar = np.zeros((len(xs), len(x)))
    for i in range(len(xs)):
        for j in range(len(x)):
            kar[i][j] = calc_kernel(xs[i], x[j])

    return svc.predict(kar)

In [13]:
fit(x_train, y_train)
y_pred = predict(x_test, x_train)
f1_score(y_test, y_pred, average="weighted")

0.9473684210526315